In [45]:
file = open(r"C:\Users\avish\OneDrive\Desktop\sentence.txt.txt")
document=file.read()
file.close()

In [55]:
document

'API stands for application programmable interface ,APIs are used as an interface between frontend and backend . API fetches data from databases and loads it into the frontend . API is used as medium of communication between Stwom systems'

In [56]:
sentences = document.split(".")
sentences

['API stands for application programmable interface ,APIs are used as an interface between frontend and backend ',
 ' API fetches data from databases and loads it into the frontend ',
 ' API is used as medium of communication between Stwom systems']

In [57]:
clean_sent=[]
for sentence in sentences:
    sentence=re.sub(r"^A-Za-z0-9"," ",sentence.lower())
    #sentence=re.sub(r"(!?|:^)\w (?:$|)"," ",sentence.lower()).strip
    clean_sent.append(sentence)

In [58]:
clean_sent

['api stands for application programmable interface ,apis are used as an interface between frontend and backend ',
 ' api fetches data from databases and loads it into the frontend ',
 ' api is used as medium of communication between stwom systems']

In [62]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer =Tokenizer()
tokenizer.fit_on_texts(clean_sent)
sequence = tokenizer.texts_to_sequences(clean_sent)
print(sequence)


[[1, 8, 9, 10, 11, 2, 12, 13, 3, 4, 14, 2, 5, 6, 7, 15], [1, 16, 17, 18, 19, 7, 20, 21, 22, 23, 6], [1, 24, 3, 4, 25, 26, 27, 5, 28, 29]]


In [66]:
ix_to_word ={}
word_to_ix={}

for i,seq in enumerate(sequence):
    
    words_in_sentence = clean_sent[i].split()
    
    for j,value in enumerate(seq):
        ix_to_word[value]=words_in_sentence[j]
        word_to_ix[words_in_sentence[j]]=value
    
    
ix_to_word
    
    

{1: 'api',
 8: 'stands',
 9: 'for',
 10: 'application',
 11: 'programmable',
 2: 'interface',
 12: ',apis',
 13: 'are',
 3: 'used',
 4: 'as',
 14: 'an',
 5: 'between',
 6: 'frontend',
 7: 'and',
 15: 'backend',
 16: 'fetches',
 17: 'data',
 18: 'from',
 19: 'databases',
 20: 'loads',
 21: 'it',
 22: 'into',
 23: 'the',
 24: 'is',
 25: 'medium',
 26: 'of',
 27: 'communication',
 28: 'stwom',
 29: 'systems'}

In [81]:
vocab_size = len(tokenizer.word_index) + 1
emb_size = 10
context_size=2

contexts = []
targets = []

for seq in sequence:
    for i in range(context_size,len(seq) - context_size):
        target=seq[i]
        context= [seq[i- 2],seq[i-1],seq[i+1],seq[i+2]]
        contexts.append(context)
        targets.append(target)
        


In [82]:
len(targets)
len(contexts)

25

In [83]:
for i in range(5):
    words=[]
    target=ix_to_word[targets[i]]
    
    for j in contexts[i]:
        words.append(ix_to_word[j])
        
    print(words , "--->",target)
        
        

['api', 'stands', 'application', 'programmable'] ---> for
['stands', 'for', 'programmable', 'interface'] ---> application
['for', 'application', 'interface', ',apis'] ---> programmable
['application', 'programmable', ',apis', 'are'] ---> interface
['programmable', 'interface', 'are', 'used'] ---> ,apis


In [84]:
X=np.array(contexts)
Y=np.array(targets)

In [89]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda,Embedding,Dense

model=Sequential([
    Embedding(input_dim=vocab_size, output_dim=emb_size,input_length=2*context_size),
    Lambda(lambda x:tf.reduce_mean(x,axis=1)),
    Dense(256,activation="relu"),
    Dense(512,activation="relu"),
    Dense(vocab_size,activation="softmax")
])

In [91]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, epochs=80)

Epoch 1/80
1/1 [==============================] - 1s 639ms/step - loss: 3.4008 - accuracy: 0.0000e+00
Epoch 2/80
1/1 [==============================] - 0s 0s/step - loss: 3.3948 - accuracy: 0.2000
Epoch 3/80
1/1 [==============================] - 0s 5ms/step - loss: 3.3891 - accuracy: 0.1600
Epoch 4/80
1/1 [==============================] - 0s 16ms/step - loss: 3.3831 - accuracy: 0.1200
Epoch 5/80
1/1 [==============================] - 0s 11ms/step - loss: 3.3763 - accuracy: 0.1200
Epoch 6/80
1/1 [==============================] - 0s 0s/step - loss: 3.3684 - accuracy: 0.1200
Epoch 7/80
1/1 [==============================] - 0s 3ms/step - loss: 3.3594 - accuracy: 0.1200
Epoch 8/80
1/1 [==============================] - 0s 17ms/step - loss: 3.3490 - accuracy: 0.1200
Epoch 9/80
1/1 [==============================] - 0s 15ms/step - loss: 3.3372 - accuracy: 0.1200
Epoch 10/80
1/1 [==============================] - 0s 0s/step - loss: 3.3237 - accuracy: 0.1200
Epoch 11/80
1/1 [===============

In [101]:
test_sentences = ["api stands application programmable",
                   "api are as an",
                  "api fetches from databases"
                  
                  ]

In [102]:
for sent in test_sentences:
    test_words = sent.split(" ")
#     print(test_words)
    x_test =[]
    for i in test_words:
        x_test.append(word_to_ix.get(i))
    x_test = np.array([x_test])
#     print(x_test)
    
    pred = model.predict(x_test)
    pred = np.argmax(pred[0])
    print("pred ", test_words, "\n=", ix_to_word.get(pred),"\n\n")

1/1 [==============================] - 0s 21ms/step
pred  ['api', 'stands', 'application', 'programmable'] 
= for 


1/1 [==============================] - 0s 19ms/step
pred  ['api', 'are', 'as', 'an'] 
= used 


1/1 [==============================] - 0s 20ms/step
pred  ['api', 'fetches', 'from', 'databases'] 
= data 


